<a href="https://colab.research.google.com/github/vice2city/generative_model_study/blob/main/Week1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.utils.data as data
import torchvision

In [ ]:
torch.manual_seed(1)

EPOCH = 1
BATCH_SIZE = 50
LR = 0.001
DOWNLOAD_MNIST = True

In [ ]:
train_data = torchvision.datasets.MNIST(
    root='./mnist/',
    train=True,
    transform=torchvision.transforms.ToTensor(),
    download=DOWNLOAD_MNIST
)

print(train_data.data.size())
print(train_data.targets.size())


torch.Size([60000, 28, 28])
torch.Size([60000])


In [ ]:
test_data = torchvision.datasets.MNIST(root='./mnist/', train=False)
train_loader = data.DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)
test_x = torch.unsqueeze(test_data.data, dim=1).type(torch.float32)[:2000]/255.
test_y = test_data.targets[:2000]

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(
                in_channels=1,
                out_channels=16,
                kernel_size=5,
                stride=1,
                padding=2
            ),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(16, 32, 5, 1, 2),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.out = nn.Linear(32 * 7 * 7, 10)
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0), -1)
        return self.out(x)

cnn = CNN()
print(cnn)

CNN(
  (conv1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (out): Linear(in_features=1568, out_features=10, bias=True)
)


In [ ]:
optimizer = torch.optim.Adam(cnn.parameters(), lr=LR)
loss_func = nn.CrossEntropyLoss()

for epoch in range(EPOCH):
    for step, (b_x, b_y) in enumerate(train_loader):
        output = cnn(b_x)[0]
        loss = loss_func(output, b_y[0])
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if step % 50 == 0:
            test_output = cnn(test_x)
            pred_y = torch.max(test_output, 1)[1].data.numpy()
            accuracy = float((pred_y == test_y.data.numpy()).astype(int).sum()) / float(test_y.size(0))
            print('Epoch: ', epoch, '| train loss: %.4f' % loss.data.numpy(), '| test accuracy: %.2f' % accuracy)


Epoch:  0 | train loss: 2.2515 | test accuracy: 0.09
Epoch:  0 | train loss: 2.2957 | test accuracy: 0.09
Epoch:  0 | train loss: 1.6676 | test accuracy: 0.42
Epoch:  0 | train loss: 0.5221 | test accuracy: 0.50
Epoch:  0 | train loss: 4.2695 | test accuracy: 0.64
Epoch:  0 | train loss: 0.5873 | test accuracy: 0.66
Epoch:  0 | train loss: 1.9073 | test accuracy: 0.68
Epoch:  0 | train loss: 2.3103 | test accuracy: 0.70
Epoch:  0 | train loss: 0.9738 | test accuracy: 0.75
Epoch:  0 | train loss: 0.0045 | test accuracy: 0.84
Epoch:  0 | train loss: 0.9623 | test accuracy: 0.83
Epoch:  0 | train loss: 0.1453 | test accuracy: 0.83
Epoch:  0 | train loss: 0.5448 | test accuracy: 0.84
Epoch:  0 | train loss: 0.0001 | test accuracy: 0.86
Epoch:  0 | train loss: 0.0216 | test accuracy: 0.88
Epoch:  0 | train loss: 0.8890 | test accuracy: 0.85
Epoch:  0 | train loss: 0.0166 | test accuracy: 0.88
Epoch:  0 | train loss: 0.0134 | test accuracy: 0.86
Epoch:  0 | train loss: 0.2028 | test accuracy

In [ ]:
test_output = cnn(test_x[:10])
pred_y = torch.max(test_output, 1)[1].data.numpy()
print(pred_y, 'prediction number')
print(test_y[:10].numpy(), 'real number')

[7 2 1 0 4 1 4 9 5 9] prediction number
[7 2 1 0 4 1 4 9 5 9] real number
